In [1]:
!nvidia-smi

Wed Sep 18 17:18:00 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   40C    P8               9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

# Análise de Variação Diária dos Preços de Ações

In [3]:
%%writefile variacao_diaria.cu
#include <thrust/host_vector.h>
#include <thrust/device_vector.h>
#include <thrust/transform.h>
#include <fstream>
#include <iostream>
#include <vector>
#include <string>

// Functor para calcular a diferença entre dois valores consecutivos
struct diff_functor {
    __host__ __device__
    float operator()(const float& a, const float& b) const {
        return b - a;  // Calcula (b - a), ou seja, (stocks[i+1] - stocks[i])
    }
};

// Função para carregar os preços de ações do arquivo
std::vector<float> load_stock_prices(const std::string& filename) {
    std::vector<float> prices;
    std::ifstream file(filename);
    if (!file) {
        std::cerr << "Erro ao abrir o arquivo: " << filename << std::endl;
        return prices;
    }

    float price;
    while (file >> price) {
        prices.push_back(price);
    }

    return prices;
}

int main() {
    // Carregar os preços de ações do arquivo "stocks-google.txt"
    std::vector<float> stock_prices = load_stock_prices("/content/stocks-google.txt");

    if (stock_prices.empty()) {
        std::cerr << "Nenhum dado encontrado no arquivo." << std::endl;
        return 1;
    }

    // Copiar os dados do host para o dispositivo
    thrust::device_vector<float> stocks(stock_prices.begin(), stock_prices.end());

    // Criar um vetor para armazenar as diferenças diárias (um elemento a menos)
    thrust::device_vector<float> ganho_diario(stocks.size() - 1);

    // Usar thrust::transform para calcular as diferenças
    thrust::transform(stocks.begin(), stocks.end() - 1, stocks.begin() + 1, ganho_diario.begin(), diff_functor());

    // Exibir o vetor de ganho_diario
    std::cout << "Diferenças diárias (ganho_diario): ";
    for (int i = 0; i < ganho_diario.size(); i++) {
        std::cout << ganho_diario[i] << " ";
    }
    std::cout << std::endl;

    return 0;
}


Writing variacao_diaria.cu


In [4]:
!nvcc -arch=sm_70 -std=c++14 variacao_diaria.cu -o variacao_diaria

In [5]:
!./variacao_diaria

Diferenças diárias (ganho_diario): 5.26001 1.5 -3.25 6.25 3.5 3.25 -1.75 5 -2.5 5.5 -5.25 6.25 1.5 2.5 2.25 -5.25 -1.5 3.75 5.5 1.25 2.5 2.5 4.25 2.25 5.25 2.25 2.5 2.75 2.5 2.25 2.25 2.25 3.5 2.25 2.5 2.25 3.5 2.25 2.25 4.5 3.25 1.25 3.25 3.25 2.25 3.25 2.5 2.5 3.25 2.5 2.75 2.5 2.25 2.75 2.5 2.5 2.25 3.5 2.25 2.5 2.25 3.5 3.25 2.5 3.25 2.75 2.75 3.25 3.25 2.75 3.5 2.5 2.5 2.25 3.25 2.25 4.25 2.5 3.75 3.25 2.25 3.25 2.5 3.25 3.25 2.5 3.75 2.25 2.5 2.5 3.25 2.25 3.5 3.25 2.75 3.75 3.75 2.5 3.25 3.5 2.25 3.5 3.25 3.5 2.75 3.5 3.5 2.5 3.25 3.25 3.5 2.5 3.25 2.5 3.25 3.5 2.5 3.25 2.25 3.25 3.25 3.25 2.75 2.5 3.5 2.75 2.75 2.75 2.75 3.75 3.25 3.5 2.5 3.25 3.5 2.75 2.5 3.25 3.25 2.5 3.75 2.25 3.5 3.25 3.5 2.75 3.5 3.5 3.25 2.25 3.25 3.25 3.25 3.25 2.5 3.75 3.25 3.5 3.5 2.5 3.25 3.25 3.25 3.25 2.5 3.25 3.5 3.25 3.5 3.25 2.75 3.5 3.25 3.25 3.25 3.25 2.5 3.25 2.25 3.5 3.5 3.25 3.25 3.5 3.25 2.75 3.75 3.25 3.5 3.25 2.75 3.5 3.25 3.5 2.25 3.25 3.25 3.25 3.25 3.25 3.25 3.25 3.25 3.25 3.25 3.25 3.

# Contagem de Dias com Aumento no Preço das Ações

In [9]:
%%writefile verifica_positivo.cu
#include <thrust/host_vector.h>
#include <thrust/device_vector.h>
#include <thrust/transform.h>
#include <thrust/count.h>
#include <iostream>
#include <fstream>
#include <vector>
#include <string>

// Functor para calcular a diferença entre dois valores consecutivos
struct diff_functor {
    __host__ __device__
    float operator()(const float& a, const float& b) const {
        return b - a;  // Calcula (b - a), ou seja, (stocks[i+1] - stocks[i])
    }
};

// Functor para verificar se o valor é positivo
struct is_positive {
    __host__ __device__
    bool operator()(const float& x) const {
        return x > 0;  // Retorna true se o valor for positivo
    }
};

// Função para carregar os preços de ações do arquivo
std::vector<float> load_stock_prices(const std::string& filename) {
    std::vector<float> prices;
    std::ifstream file(filename);
    if (!file) {
        std::cerr << "Erro ao abrir o arquivo: " << filename << std::endl;
        return prices;
    }

    float price;
    while (file >> price) {
        prices.push_back(price);
    }

    return prices;
}

int main() {
    // Carregar os preços de ações do arquivo "stocks-google.txt"
    std::vector<float> stock_prices = load_stock_prices("/content/stocks-google.txt");

    if (stock_prices.empty()) {
        std::cerr << "Nenhum dado encontrado no arquivo." << std::endl;
        return 1;
    }

    // Copiar os dados do host para o dispositivo
    thrust::device_vector<float> stocks(stock_prices.begin(), stock_prices.end());

    // Criar um vetor para armazenar as diferenças diárias (um elemento a menos)
    thrust::device_vector<float> ganho_diario(stocks.size() - 1);

    // Usar thrust::transform para calcular as diferenças
    thrust::transform(stocks.begin(), stocks.end() - 1, stocks.begin() + 1, ganho_diario.begin(), diff_functor());

    // Contar quantos dias o preço das ações subiu
    int dias_com_aumento = thrust::count_if(ganho_diario.begin(), ganho_diario.end(), is_positive());

    std::cout << "Número de dias com aumento no preço das ações: " << dias_com_aumento << std::endl;

    return 0;
}


Overwriting verifica_positivo.cu


In [10]:
!nvcc -arch=sm_70 -std=c++14 verifica_positivo.cu -o verifica_positivo

In [11]:
!./verifica_positivo

Número de dias com aumento no preço das ações: 3056


In [25]:
%%writefile variancia_com_iteradores.cu
#include <thrust/host_vector.h>
#include <thrust/device_vector.h>
#include <thrust/transform.h>
#include <thrust/reduce.h>
#include <thrust/iterator/constant_iterator.h>
#include <iostream>
#include <fstream>
#include <vector>
#include <string>
#include <chrono>  // Para medir tempo na CPU
#include <cuda_runtime.h>  // Para medir tempo na GPU

// Functor para calcular a variância (xi - μ)^2
struct variance_functor {
    __host__ __device__
    float operator()(const float& x, const float& mean) const {
        return (x - mean) * (x - mean);  // (xi - μ)^2
    }
};

// Função para carregar os preços de ações do arquivo
std::vector<float> load_stock_prices(const std::string& filename) {
    std::vector<float> prices;
    std::ifstream file(filename);
    if (!file) {
        std::cerr << "Erro ao abrir o arquivo: " << filename << std::endl;
        return prices;
    }

    float price;
    while (file >> price) {
        prices.push_back(price);
    }

    return prices;
}

int main() {
    // Medir tempo para carregar os preços de ações
    auto start_load = std::chrono::high_resolution_clock::now();
    std::vector<float> stock_prices = load_stock_prices("/content/stocks-google.txt");
    auto end_load = std::chrono::high_resolution_clock::now();
    std::chrono::duration<double> elapsed_load = end_load - start_load;
    std::cout << "=====================" << std::endl;
    std::cout << "Tempo para carregar preços de ações: " << elapsed_load.count() << " segundos\n";
    std::cout << "=====================" << std::endl;

    if (stock_prices.empty()) {
        std::cerr << "Nenhum dado encontrado no arquivo." << std::endl;
        return 1;
    }

    // Copiar os dados do host para o dispositivo
    thrust::device_vector<float> stocks(stock_prices.begin(), stock_prices.end());

    // Calcular a média dos preços de ações
    float soma_precos = thrust::reduce(stocks.begin(), stocks.end(), 0.0f, thrust::plus<float>());
    float media = soma_precos / stocks.size();

    // Inicializando eventos CUDA para medir tempo de GPU
    cudaEvent_t start, stop;
    cudaEventCreate(&start);
    cudaEventCreate(&stop);

    // Criar um vetor para armazenar a variância
    thrust::device_vector<float> variancia(stocks.size());

    // Usar iteradores dinâmicos para evitar a criação de um vetor de médias
    cudaEventRecord(start);
    thrust::transform(stocks.begin(), stocks.end(),
                      thrust::constant_iterator<float>(media),
                      variancia.begin(),
                      variance_functor());
    cudaEventRecord(stop);
    cudaEventSynchronize(stop);
    float ms_transform = 0;
    cudaEventElapsedTime(&ms_transform, start, stop);
    std::cout << "Tempo para calcular a variância: " << ms_transform << " ms\n";
    std::cout << "=====================" << std::endl;

    // Somar a variância
    float soma_variancia = thrust::reduce(variancia.begin(), variancia.end(), 0.0f, thrust::plus<float>());

    // Calcular a variância final
    float variancia_final = soma_variancia / stocks.size();
    std::cout << "Variância final: " << variancia_final << std::endl;

    // Liberar eventos CUDA
    cudaEventDestroy(start);
    cudaEventDestroy(stop);

    return 0;
}


Overwriting variancia_com_iteradores.cu


In [26]:
!nvcc -arch=sm_70 -std=c++14 variancia_com_iteradores.cu -o variancia_com_iteradores

In [27]:
!./variancia_com_iteradores

Tempo para carregar preços de ações: 0.00135161 segundos
Tempo para calcular a variância: 0.028672 ms
Variância final: 123792


# Cálculo do Aumento Médio nos Dias em que o Preço Subiu

In [18]:
%%writefile aumento_medio.cu
#include <thrust/host_vector.h>
#include <thrust/device_vector.h>
#include <thrust/transform.h>
#include <thrust/count.h>
#include <thrust/replace.h>
#include <thrust/reduce.h>
#include <iostream>
#include <fstream>
#include <vector>
#include <string>
#include <chrono>  // Para medir tempo na CPU
#include <cuda_runtime.h>  // Para medir tempo na GPU

// Functor para calcular a diferença entre dois valores consecutivos
struct diff_functor {
    __host__ __device__
    float operator()(const float& a, const float& b) const {
        return b - a;  // Calcula (b - a), ou seja, (stocks[i+1] - stocks[i])
    }
};

// Functor para verificar se o valor é positivo
struct is_positive {
    __host__ __device__
    bool operator()(const float& x) const {
        return x > 0;  // Retorna true se o valor for positivo
    }
};

// Função para carregar os preços de ações do arquivo
std::vector<float> load_stock_prices(const std::string& filename) {
    std::vector<float> prices;
    std::ifstream file(filename);
    if (!file) {
        std::cerr << "Erro ao abrir o arquivo: " << filename << std::endl;
        return prices;
    }

    float price;
    while (file >> price) {
        prices.push_back(price);
    }

    return prices;
}

int main() {
    // Medir tempo para carregar os preços de ações
    auto start_load = std::chrono::high_resolution_clock::now();
    std::vector<float> stock_prices = load_stock_prices("/content/stocks-google.txt");
    auto end_load = std::chrono::high_resolution_clock::now();
    std::chrono::duration<double> elapsed_load = end_load - start_load;
    std::cout << "=====================" << std::endl;
    std::cout << "Tempo para carregar preços de ações: " << elapsed_load.count() << " segundos\n";
    std::cout << "=====================" << std::endl;

    if (stock_prices.empty()) {
        std::cerr << "Nenhum dado encontrado no arquivo." << std::endl;
        return 1;
    }

    // Copiar os dados do host para o dispositivo
    thrust::device_vector<float> stocks(stock_prices.begin(), stock_prices.end());

    // Criar um vetor para armazenar as diferenças diárias (um elemento a menos)
    thrust::device_vector<float> ganho_diario(stocks.size() - 1);

    // Inicializando eventos CUDA para medir tempo de GPU
    cudaEvent_t start, stop;
    cudaEventCreate(&start);
    cudaEventCreate(&stop);

    // Usar thrust::transform para calcular as diferenças
    cudaEventRecord(start);
    thrust::transform(stocks.begin(), stocks.end() - 1, stocks.begin() + 1, ganho_diario.begin(), diff_functor());
    cudaEventRecord(stop);
    cudaEventSynchronize(stop);
    float ms_transform = 0;
    cudaEventElapsedTime(&ms_transform, start, stop);
    std::cout << "Tempo para calcular diferenças diárias: " << ms_transform << " ms\n";
    std::cout << "=====================" << std::endl;

    // Contar quantos dias o preço das ações subiu
    cudaEventRecord(start);
    int dias_com_aumento = thrust::count_if(ganho_diario.begin(), ganho_diario.end(), is_positive());
    cudaEventRecord(stop);
    cudaEventSynchronize(stop);
    float ms_count = 0;
    cudaEventElapsedTime(&ms_count, start, stop);
    std::cout << "Tempo para contar dias com aumento no preço: " << ms_count << " ms\n";
    std::cout << "=====================" << std::endl;

    // Substituir valores negativos por zero
    cudaEventRecord(start);
    thrust::replace_if(ganho_diario.begin(), ganho_diario.end(), thrust::placeholders::_1 <= 0, 0.0f);
    cudaEventRecord(stop);
    cudaEventSynchronize(stop);
    float ms_replace = 0;
    cudaEventElapsedTime(&ms_replace, start, stop);
    std::cout << "Tempo para substituir valores negativos: " << ms_replace << " ms\n";
    std::cout << "=====================" << std::endl;

    // Calcular a soma dos valores positivos
    cudaEventRecord(start);
    float soma_aumentos = thrust::reduce(ganho_diario.begin(), ganho_diario.end(), 0.0f, thrust::plus<float>());
    cudaEventRecord(stop);
    cudaEventSynchronize(stop);
    float ms_reduce = 0;
    cudaEventElapsedTime(&ms_reduce, start, stop);
    std::cout << "Tempo para calcular soma dos aumentos: " << ms_reduce << " ms\n";
    std::cout << "=====================" << std::endl;

    // Calcular a média dos aumentos
    float media_aumento = soma_aumentos / dias_com_aumento;
    std::cout << "Número de dias com aumento no preço das ações: " << dias_com_aumento << std::endl;
    std::cout << "Soma dos aumentos: " << soma_aumentos << std::endl;
    std::cout << "Média dos aumentos: " << media_aumento << std::endl;
    std::cout << "=====================" << std::endl;

    // Liberar eventos CUDA
    cudaEventDestroy(start);
    cudaEventDestroy(stop);

    return 0;
}


Overwriting aumento_medio.cu


In [19]:
!nvcc -arch=sm_70 -std=c++14 aumento_medio.cu -o aumento_medio

In [20]:
!./aumento_medio

Tempo para carregar preços de ações: 0.00136044 segundos
Tempo para calcular diferenças diárias: 0.028576 ms
Tempo para contar dias com aumento no preço: 0.137664 ms
Tempo para substituir valores negativos: 0.049184 ms
Tempo para calcular soma dos aumentos: 0.085696 ms
Número de dias com aumento no preço das ações: 3056
Soma dos aumentos: 9634.08
Média dos aumentos: 3.15251


# Discussão de Resultados

* O uso da GPU e das bibliotecas Thrust permitiu que todas essas operações fossem realizadas com tempos de execução extremamente baixos. Cada uma das operações críticas levou apenas frações de milissegundo, demonstrando a eficácia da paralelização para grandes volumes de dados financeiros.

* Esse tipo de análise pode ser muito útil em cenários em que há grandes quantidades de dados a serem processados, como no mercado financeiro, onde o uso da GPU pode acelerar significativamente o processamento e a análise de dados históricos.

Eu fiz duas versões da parte final da aula, uma usando o  `thrust::constant_iterator`e outra sem. Ele de fato diminui o tempo que demora para rodar aquela parte do codigo, mas como não temos dados muito grande ou mais importante, uma conta complexa, não da para perceber muita diferença. Mas em tese era para ela permitir o acesso a dados sob demanda, sem gastar memória para armazenar grandes vetores repetitivos, tornando o código mais leve e eficiente.

Outra coisa sobre a função de transformação, ela ajuda a paralelizar o processo, assim podendo tirar o máximo da memória da GPU (só ganhos, hehehe).

Em suma, essas duas técnicas resultam em tempos de execução baixissimos com o menor consumo de memória possível, logo se tornam cruciais para o trabalho com grandes volumes de dados.